# Error analysis notebook (test data)

## Import packages

In [1]:
import pandas as pd
import math

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from elasticsearch import Elasticsearch

import warnings
import matplotlib
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
warnings.filterwarnings("ignore")

from enum import IntEnum

import json


/home/pilmus/thesis/venv37/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Set filepaths

In [2]:
eval_result_ac = '../evaluation/resources/2020/eval_results/p_controller_train_nle_meta_9_1_train_hlevel.tsv'
eval_result_mrfr = '../evaluation/resources/2020/eval_results/mrfr_train_META.tsv'
eval_result_lm_mrfr = '../evaluation/resources/2020/eval_results/mrfr_train_LM_train.tsv'
eval_result_lm =  '../evaluation/resources/2020/eval_results/lambdamart2020_train_seed0hlevel_random_state=0.tsv'
eval_result_lm_aug07 =  '../evaluation/resources/2020/eval_results/lambdamart2020_train_aug0.7_seed0_random_state=0.tsv'


## Prepare dataframes

In [3]:
# eval result dfs
eacdf = pd.read_csv(eval_result_ac, sep='\t', names=['key', 'qid', 'value'])
eacdf = eacdf.pivot(index='qid', columns='key', values='value')

emrfrdf = pd.read_csv(eval_result_mrfr, sep='\t', names=['key', 'qid', 'value'])
emrfrdf = emrfrdf.pivot(index='qid', columns='key', values='value')

elmmrfrdf = pd.read_csv(eval_result_lm_mrfr, sep='\t', names=['key', 'qid', 'value'])
elmmrfrdf = elmmrfrdf.pivot(index='qid', columns='key', values='value')

elmdf = pd.read_csv(eval_result_lm, sep='\t', names=['key', 'qid', 'value'])
elmdf = elmdf.pivot(index='qid', columns='key', values='value')

elmaug07df = pd.read_csv(eval_result_lm_aug07, sep='\t', names=['key', 'qid', 'value'])
elmaug07df = elmaug07df.pivot(index='qid', columns='key', values='value')



In [4]:
eacdf.mean()

key
difference    0.434440
disparity     1.753156
relevance     1.252646
dtype: float64

In [5]:
emrfrdf.mean()

key
difference    0.437316
disparity     1.759714
relevance     1.254487
dtype: float64

In [6]:
elmmrfrdf.mean()

key
difference    0.472034
disparity     1.741128
relevance     1.227835
dtype: float64

In [7]:
elmdf.mean()

key
difference    0.740297
disparity     1.970017
relevance     1.208148
dtype: float64

In [8]:
elmaug07df.mean()

key
difference    0.702419
disparity     1.895508
relevance     1.189833
dtype: float64